In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
x_train = x_train.reshape(50000, -1)
x_test = x_test.reshape(10000, -1)

In [4]:
temp1 = x_train
temp2 = x_test

In [5]:
# Standardizing the data to apply PCA later
from sklearn.preprocessing import StandardScaler
x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)

In [6]:
x_train.shape

(50000, 3072)

In [7]:
# Applying PCA
from sklearn.decomposition import PCA
pca = PCA(.98)
pca.fit(x_train)

PCA(n_components=0.98)

In [8]:
pca.n_components_

448

In [9]:
pca_x_train = pca.transform(x_train)

In [10]:
pca_x_test = pca.transform(x_test)

In [11]:
pca_x_train.shape

(50000, 448)

In [12]:
pca_x_test.shape

(10000, 448)

In [13]:
from sklearn.linear_model import LogisticRegression
import sklearn

In [14]:
logisticRegr = LogisticRegression()

In [15]:
logisticRegr.fit(pca_x_train, y_train)

/opt/miniconda3/envs/cs188/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/miniconda3/envs/cs188/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [16]:
pca_predicts = logisticRegr.predict(pca_x_test)

In [17]:
sklearn.metrics.accuracy_score(pca_predicts, y_test)

0.4086

In [18]:
model = keras.Sequential(
    [
        layers.Dense(448, activation="relu", name="layer1"),
        layers.Dense(300, activation="relu", name="layer2"),
        layers.Dense(100, activation="relu", name="layer3"),
        layers.Dense(64, activation="relu", name="layer4"),
        layers.Dense(10, activation="softmax", name="layer5"),
    ])

In [19]:
loss = tf.keras.losses.CategoricalCrossentropy() # A parameter to do softmax implicitly from_logits
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = optimizer, loss=loss, metrics = ['accuracy'])

In [20]:
def convertLabelsToArray(labels):
    newLabel = []
    for label in labels:
        newList= [0] * 10
        newList[label[0]] = 1
        newLabel.append(newList)
    return np.array(newLabel)

In [21]:
newY_train = convertLabelsToArray(y_train)
newY_test = convertLabelsToArray(y_test)

In [22]:
model.fit(pca_x_train, newY_train, epochs = 5)

Epoch 1/5
1563/1563 [==============================] - 3s 2ms/step - loss: 1.7788 - accuracy: 0.3694
Epoch 2/5
1563/1563 [==============================] - 3s 2ms/step - loss: 1.3219 - accuracy: 0.5344
Epoch 3/5
1563/1563 [==============================] - 3s 2ms/step - loss: 1.1134 - accuracy: 0.6087
Epoch 4/5
1563/1563 [==============================] - 3s 2ms/step - loss: 0.9408 - accuracy: 0.6624
Epoch 5/5
1563/1563 [==============================] - 3s 2ms/step - loss: 0.7794 - accuracy: 0.7217


m

In [23]:
model.predict(pca_x_train)[3]

array([2.2522370e-04, 3.6914389e-05, 7.7976431e-03, 8.5352901e-03,
       9.5713961e-01, 6.2506883e-03, 1.8154109e-02, 1.7303457e-03,
       2.3633771e-05, 1.0646934e-04], dtype=float32)

In [24]:
y_train[3]

array([4], dtype=uint8)

In [25]:
def convertPredictionsToLabels(predictions):
    return [np.argmax(row) for row in predictions]

In [26]:
testPredictions = model.predict(pca_x_test)
testPredictedLabels = convertPredictionsToLabels(testPredictions)
accuracy_score(testPredictedLabels, y_test)

0.5255